In [3]:
import yfinance as yf 
from yfinance import EquityQuery
import numpy as np
import pandas as pd

size = 250
rsi_window = 14
roc_window = 5
volatility_window = 30

In [4]:
# Defining a screen for US stocks in specific sectors with certain criteria -- presecreen for options volatility 
screen = EquityQuery("and",[EquityQuery("is-in", ["region", "us"]), EquityQuery("is-in", ["sector",
    "Technology",
    "Financial Services",
    "Consumer Cyclical",
    "Communication Services",
    "Basic Materials",
    "Industrials"
]), EquityQuery("GTE", ["eodprice",0.50])
,   EquityQuery("GTE", ["avgdailyvol3m", 100000]), 
EquityQuery("LTE", ["short_percentage_of_float.value", 100])])

screener = yf.screen(screen,sortField="avgdailyvol3m", sortAsc=False, size = size)

stocks = screener.get('quotes', [])


In [5]:
stocks = pd.DataFrame(stocks)
stocks = stocks.drop(columns=[
    'esgPopulated',
    'cryptoTradeable',
    'hasPrePostMarketData',
    'firstTradeDateMilliseconds',
    'priceHint',
    'regularMarketDayHigh',
    'regularMarketDayRange',
    'regularMarketDayLow',
    'regularMarketPreviousClose',
    'messageBoardId',
    'fullExchangeName',
    'financialCurrency',
    'regularMarketOpen',
    'averageDailyVolume3Month',
    'averageDailyVolume10Day',
    'corporateActions',
    'fiftyTwoWeekLowChange',
    'fiftyTwoWeekLowChangePercent',
    'fiftyTwoWeekRange',
    'fiftyTwoWeekHighChange',
    'fiftyTwoWeekHighChangePercent',
    'fiftyTwoWeekChangePercent',
    'earningsTimestamp',
    'earningsTimestampStart',
    'earningsTimestampEnd',
    'earningsCallTimestampStart',
    'earningsCallTimestampEnd',
    'dividendRate',
    'trailingAnnualDividendYield',
    'marketState',
    'epsCurrentYear',
    'fiftyDayAverage',
    'fiftyDayAverageChange',
    'fiftyDayAverageChangePercent',
    'twoHundredDayAverage',
    'twoHundredDayAverageChange',
    'twoHundredDayAverageChangePercent','dividendYield','gmtOffSetMilliseconds','bid','ask','currency', 'quoteType','language','typeDisp','quoteSourceName','triggerable','customPriceAlertConfidence','sourceInterval','exchangeDataDelayedBy','exchangeTimezoneName','exchangeTimezoneShortName',
 'bidSize','askSize','market','isEarningsDateEstimate','trailingAnnualDividendRate','regularMarketTime','priceEpsCurrentYear','epsForward','forwardPE','tradeable','displayName','longName'
]
)
df = stocks

In [6]:
order = [
    # Identification
    'symbol',
    'shortName',
    'exchange',
    'region',
    
    # Market Price & Change
    'regularMarketPrice',
    'regularMarketChange',
    'regularMarketChangePercent',
    'regularMarketVolume',
    
    # Valuation Metrics
    'marketCap',
    'sharesOutstanding',
    'bookValue',
    'priceToBook',
    'trailingPE',
    'epsTrailingTwelveMonths',
    'averageAnalystRating',
    
    # Price Range
    'fiftyTwoWeekHigh',
    'fiftyTwoWeekLow',
    
    # Dividend Info
    'dividendDate'
]
df = df[order]
df


,symbol,shortName,exchange,region,regularMarketPrice,regularMarketChange,regularMarketChangePercent,regularMarketVolume,marketCap,sharesOutstanding,bookValue,priceToBook,trailingPE,epsTrailingTwelveMonths,averageAnalystRating,fiftyTwoWeekHigh,fiftyTwoWeekLow,dividendDate
0,NVDA,NVIDIA Corporation,NMS,US,139.19,4.380005,3.249021,367415229,3394510258176,24387600384,34.304,4.057544,47.343536,2.94,1.5 - Strong Buy,153.130,86.620,1.743552e+09
1,F,Ford Motor Company,NYQ,US,10.22,0.050000,0.491644,96699350,40640339968,3905700096,11.225,0.910468,8.176001,1.25,3.0 - Hold,14.850,8.440,1.748822e+09
2,TSLA,"Tesla, Inc.",NMS,US,358.43,1.529999,0.428691,87309442,1154488729600,3220960000,23.184,15.460231,204.817140,1.75,2.6 - Hold,488.540,167.410,NaN
3,LCID,"Lucid Group, Inc.",NMS,US,2.41,-0.220000,-8.365020,261178376,7351127040,3050259968,1.044,2.308429,NaN,-1.19,3.1 - Hold,4.430,1.930,NaN
4,PLTR,Palantir Technologies Inc.,NMS,US,122.32,-1.440002,-1.163544,62590548,288664190976,2262909952,2.299,53.205740,509.666700,0.24,3.1 - Hold,133.490,21.005,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,ZETA,Zeta Global Holdings Corp.,NYQ,US,13.09,-0.410000,-3.037036,3224952,3083270912,211448992,2.866,4.567341,NaN,-0.25,1.6 - Buy,38.200,10.690,NaN
246,BZ,KANZHUN LIMITED - American Depo,NMS,US,17.70,0.130001,0.739904,2027174,7795434496,372296000,36.412,0.486104,31.052633,0.57,1.7 - Buy,21.990,10.570,1.703117e+09
247,JCI,Johnson Controls International,NYQ,US,102.14,0.650002,0.640459,4489709,67212918784,658046976,24.018,4.252644,30.858006,3.31,2.0 - Buy,102.355,64.310,1.744848e+09
248,SG,"Sweetgreen, Inc.",NYQ,US,12.76,-0.740000,-5.481480,3964127,1501622400,105788000,3.678,3.469277,NaN,-0.77,1.9 - Buy,45.120,12.370,NaN


In [11]:
# Get the price history for the stocks, to do RSI, or MACD, options volatility, and kalman filter acceleration(or see tview workaround)
stocklist = df.symbol.tolist()
stocklist

# Get the price history for the stocks
price_data = yf.download(stocklist, period='14d', interval='1d')
price_data = price_data['Close']

# Calculate the daily returns
change_data = price_data[stocklist]/price_data[stocklist].shift(1)-1

# RSI
change_data
def rsi(price_data, period=rsi_window):
    delta = change_data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))
rsi_data = rsi(price_data)
rsi_data

rsi = round(rsi_data.iloc[-1],1)
df['RSI'] = rsi.values


## Rate of change 
window=5 
recent_avg = change_data.rolling(window=roc_window).mean()
earlier_avg = change_data.shift(window).rolling(window=roc_window).mean()

# Acceleration: difference between recent and earlier averages
accel = (recent_avg - earlier_avg) * 100
accel = round(accel.iloc[-1], 2)
df['ROC'] = accel.values


# Apply filters

filterdf = df[(df['RSI'] < 60) & (df['ROC'] > 1) ] # Filter for RSI < 60

filterdf



[*********************100%***********************]  250 of 250 completed
/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_39594/2240806638.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RSI'] = rsi.values
/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_39594/2240806638.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ROC'] = accel.values


,symbol,shortName,exchange,region,regularMarketPrice,regularMarketChange,regularMarketChangePercent,regularMarketVolume,marketCap,sharesOutstanding,bookValue,priceToBook,trailingPE,epsTrailingTwelveMonths,averageAnalystRating,fiftyTwoWeekHigh,fiftyTwoWeekLow,dividendDate,RSI,ROC
0,NVDA,NVIDIA Corporation,NMS,US,139.1900,4.380005,3.249021,367415229,3394510258176,24387600384,34.304,4.057544,47.343536,2.94,1.5 - Strong Buy,153.13,86.620,1.743552e+09,45.8,1.64
2,TSLA,"Tesla, Inc.",NMS,US,358.4300,1.529999,0.428691,87309442,1154488729600,3220960000,23.184,15.460231,204.817140,1.75,2.6 - Hold,488.54,167.410,NaN,43.3,2.17
4,PLTR,Palantir Technologies Inc.,NMS,US,122.3200,-1.440002,-1.163544,62590548,288664190976,2262909952,2.299,53.205740,509.666700,0.24,3.1 - Hold,133.49,21.005,NaN,47.1,1.80
5,PLUG,"Plug Power, Inc.",NCM,US,0.9264,-0.000100,-0.010795,155566955,998974144,1078339968,1.900,0.487579,NaN,-2.43,2.9 - Hold,3.55,0.690,NaN,48.1,4.19
8,AAL,"American Airlines Group, Inc.",NMS,US,11.4000,-0.030001,-0.262473,61294337,7518436352,659512000,-6.836,-1.667642,11.400000,1.00,2.1 - Buy,19.10,8.500,1.582070e+09,43.4,1.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,PAGS,PagSeguro Digital Ltd.,NYQ,US,8.5800,-0.100000,-1.152078,4794238,2562657280,178218000,49.575,0.173071,7.090909,1.21,2.1 - Buy,14.82,6.110,1.749168e+09,49.8,2.44
241,DDOG,"Datadog, Inc.",NMS,US,117.0000,0.699997,0.601889,3555664,40406298624,319497984,8.456,13.836329,248.936170,0.47,1.6 - Buy,170.08,81.630,NaN,42.3,1.51
243,SYF,Synchrony Financial,NYQ,US,58.2500,-0.060001,-0.102901,2002849,22173270016,380656992,40.361,1.443225,7.979452,7.30,2.0 - Buy,70.93,40.545,1.747267e+09,37.9,1.65
245,ZETA,Zeta Global Holdings Corp.,NYQ,US,13.0900,-0.410000,-3.037036,3224952,3083270912,211448992,2.866,4.567341,NaN,-0.25,1.6 - Buy,38.20,10.690,NaN,37.0,2.00


#"""Steps:
#1. get options volatility, prices, data 
#2. get RSI, kalman filter acceleration  (done)
#indicators in screener: filter by RSI, filter by relative options volatility (vs historical volatility, as yfinance does not have historical options volatility data)
#"""
# other:
#take steps to pull all yahoo screen companies 

In [ ]:
filterstocks = filterdf.symbol.tolist()
price_datayear = yf.download(filterstocks, period='3mo', interval='1d')
#^^^ fix 
# Historical volatiltiy calculation
# Next add the vol smile relative to HV 
def hv(price_datayear, window=volatility_window):
    log_returns = np.log(price_datayear / price_data.shift(1))
    volatility = log_returns.rolling(window=window).std() * np.sqrt(252)  # Annualized volatility
    return volatility

hv(price_datayear)

[*********************100%***********************]  103 of 103 completed


Price      Close                                           ... Volume      \
Ticker       AAL ABNB AEO AFRM ALIT AMAT AMC AMD AMZN APP  ...    VLY VRT   
Date                                                       ...              
2025-02-28   NaN  NaN NaN  NaN  NaN  NaN NaN NaN  NaN NaN  ...    NaN NaN   
2025-03-03   NaN  NaN NaN  NaN  NaN  NaN NaN NaN  NaN NaN  ...    NaN NaN   
2025-03-04   NaN  NaN NaN  NaN  NaN  NaN NaN NaN  NaN NaN  ...    NaN NaN   
2025-03-05   NaN  NaN NaN  NaN  NaN  NaN NaN NaN  NaN NaN  ...    NaN NaN   
2025-03-06   NaN  NaN NaN  NaN  NaN  NaN NaN NaN  NaN NaN  ...    NaN NaN   
...          ...  ...  ..  ...  ...  ...  ..  ..  ...  ..  ...    ...  ..   
2025-05-22   NaN  NaN NaN  NaN  NaN  NaN NaN NaN  NaN NaN  ...    NaN NaN   
2025-05-23   NaN  NaN NaN  NaN  NaN  NaN NaN NaN  NaN NaN  ...    NaN NaN   
2025-05-27   NaN  NaN NaN  NaN  NaN  NaN NaN NaN  NaN NaN  ...    NaN NaN   
2025-05-28   NaN  NaN NaN  NaN  NaN  NaN NaN NaN  NaN NaN  ...    NaN NaN   
2025-05-29   NaN  NaN NaN  NaN  NaN  NaN NaN NaN  NaN NaN  ...    NaN NaN   

Price                                         
Ticker     VVPR   W WDC WFC   X XRX YMM ZETA  
Date                                          
2025-02-28  NaN NaN NaN NaN NaN NaN NaN  NaN  
2025-03-03  NaN NaN NaN NaN NaN NaN NaN  NaN  
2025-03-04  NaN NaN NaN NaN NaN NaN NaN  NaN  
2025-03-05  NaN NaN NaN NaN NaN NaN NaN  NaN  
2025-03-06  NaN NaN NaN NaN NaN NaN NaN  NaN  
...         ...  ..  ..  ..  ..  ..  ..  ...  
2025-05-22  NaN NaN NaN NaN NaN NaN NaN  NaN  
2025-05-23  NaN NaN NaN NaN NaN NaN NaN  NaN  
2025-05-27  NaN NaN NaN NaN NaN NaN NaN  NaN  
2025-05-28  NaN NaN NaN NaN NaN NaN NaN  NaN  
2025-05-29  NaN NaN NaN NaN NaN NaN NaN  NaN  

[63 rows x 515 columns]